In [127]:
from sklearn.model_selection import train_test_split
import pandas as pd

In [128]:
overallDataset = pd.read_csv("data/MovieGenre.csv", sep=",", encoding = "ISO-8859-1")
overallDataset = overallDataset[["imdbId", "Genre"]]
# We need to drop NaNs
overallDataset = overallDataset.dropna()

In [129]:
def genre_discover(genre):
    category = None
    if "|" in genre:
        genre_split = genre.split("|")
        if (genre_split[0] == "Drama") or (genre_split[0] == "Comedy") or (genre_split[0] == "Action"):
            if (genre_split[1] == "Drama"):
                if len(genre_split) == 3:
                    category = genre_split[2]
                else:
                    category = genre_split[1]
            else:
                category = genre_split[1]
        else:
            category = genre_split[0]
    else:
        category = genre
    return category

In [130]:
overallDataset['Genre_1'] = overallDataset.apply(lambda x: genre_discover(x['Genre']), axis=1)
print("Number of Records originally: " + str(len(overallDataset["Genre_1"])))
print("Number of unique values in Genre_1 originally: " + str(len(overallDataset["Genre_1"].unique())))
#overallDataset["Genre_1"].value_counts()

Number of Records originally: 39963
Number of unique values in Genre_1 originally: 26


In [131]:
genre_counts = overallDataset["Genre_1"].value_counts()
filt_genre_counts = genre_counts[genre_counts >= 1100].index
fixedOverallDataset = overallDataset[overallDataset['Genre_1'].isin(filt_genre_counts)]

new_balanced_df = pd.DataFrame(columns=["imdbId", "Genre", "Genre_1"])
for genre in filt_genre_counts:
    cut_temp_df = fixedOverallDataset[fixedOverallDataset["Genre_1"] == genre].iloc[:1200]
    new_balanced_df = pd.concat([new_balanced_df, cut_temp_df])
    
genre_category_numbers = list(filt_genre_counts)
def label_id_add(genre):
    return int(genre_category_numbers.index(genre))

    
new_balanced_df = new_balanced_df.drop(['Genre'], axis=1)
new_balanced_df = new_balanced_df.rename(columns={"Genre_1": "Genre"})
new_balanced_df['Label'] = new_balanced_df.apply(lambda x: label_id_add(x['Genre']), axis=1)

In [132]:
new_balanced_df["Genre"].value_counts()

Drama          1200
Romance        1200
Crime          1200
Documentary    1200
Comedy         1200
Horror         1200
Adventure      1200
Animation      1200
Thriller       1200
Biography      1200
Name: Genre, dtype: int64

In [133]:
original_size = len(new_balanced_df)
train, test = train_test_split(new_balanced_df, test_size=0.1)
train_size = len(train)
test_size = len(test)

print("The new number of records was " + str(original_size) + ".")
print("The train size was " + str(train_size) + " and the test size was " + str(test_size))
print("Number of unique values Genres: " + str(len(new_balanced_df["Genre"].unique())))
assert original_size == (train_size + test_size)

The new number of records was 12000.
The train size was 10800 and the test size was 1200
Number of unique values Genres: 10


In [134]:
test.to_csv("test_labels.csv", index=False)
train.to_csv("train_labels.csv", index=False)